**Skill Level:** <code style="color : LightSkyBlue"><ins><b>Intermediate</b></ins></code>

# Google Cloud Agent Builder

## Overview

Google Cloud's **Agent Builder** feature is designed to help you build AI-powered systems, like chatbots or automated assistants, quickly and easily. It provides pre-built components (agents) for different tasks, which you can connect and customize to create sophisticated systems. You can also integrate Google's AI features like natural language processing for better performance.

## Learning objectives

- Learn how to create an agent in Agent Builder
- Learn to create an data source tool

## Prerequisites

You must have enabled the Vertex AI, Compute Engine, and Agent Builder APIs.

## Pricing

This tutorial will cost up to ~$6, pricing is based on the cost of Agent Builder and Jupyter notebooks.
For this tutorial we used a n2-standard-2 instance to run the python commands and created a Agent App [Google Cloud Pricing Calculator](https://cloud.google.com/products/calculator?hl=en).

## Get started

Create a bucket Google Cloud Bucket and upload your documents following the instructions listed [here](https://cloud.google.com/storage/docs/uploading-objects#upload-object-console). If you'd like to use the example dataset provided run the following commands:

Set your project id, location, and bucket variables.

In [ ]:
project_id='<PROJECT_ID>'
location='<LOCATION> (e.g.us-east4)'
bucket = '<BUCKET_NAME>'

Create your bucket.

In [ ]:
#make bucket
!gsutil mb -l {location} gs://{bucket}

List the URL of the files we want to download, these are from PubMed.

In [29]:
#urls list of articles
articles_urls = ['https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10781091/pdf/pone.0285645.pdf', 
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10219649/pdf/elife-86014.pdf',
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10734909/pdf/pone.0285351.pdf',
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10684592/pdf/41598_2023_Article_47655.pdf', 
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10601201/pdf/12889_2023_Article_16916.pdf',
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10516599/pdf/elife-86043.pdf',
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10620090/pdf/41586_2023_Article_6651.pdf',
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10414557/pdf/pone.0289774.pdf',
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10355333/pdf/aids-37-1565.pdf',
                 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10289397/pdf/pone.0286297.pdf']

Below we are downloading the files and uploading them to our newly create bucket. The python function 'upload_blob_from_memory' allows us to upload the files to the bucket and the for loop under it allows us to execute this function to every URL in our list.

In [ ]:
from google.cloud import storage
import os
import requests

def upload_blob_from_memory(bucket_name, contents, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)

    return print(
        f"{destination_blob_name} uploaded to {bucket_name}."
    )

for i in articles_urls:
    doc_name=i.split(r'/')[-1]
    x = requests.get(i)
    doc = x.text
    upload_blob_from_memory(bucket, doc, {doc_name})

### Create a Agent app

Agent Builder allows you to create different apps such as:
- Chatbots
- Search engine
- Recommendations bot
- Agent development

For this tutorial we are creating a GenAI agent to help guide our chatbot in certain situations. Start by searching up **'Agent Builder'** on the console. Then once you are on the page shown below click **'CREATE APP'**.

![grounding1](../../images/grounding_1.png)

Click **'SELECT'** for the app named **Agent**.

![agent_builder1](../../images/agent_builder1.png)

Give your agent a name, select your region, then click **'CREATE'**.

![agent_builder2](../../images/agent_builder2.png)

Once you have created your Agent app you will be taken to the **Agent Console**. You will see that a default agent has Already been created to help greet your users and handle specific tasks. We are going to keep this as is for now and create a **tool**. The idea is that this default agent will direct the model to another agent or a tool if the user is asking questions about the articles in our bucket.

**Optional:** you can define the goal of this agent for this tutorial we will keep it default but you can describe the goal of this agent for example: you will act as a research assistant and code interpreter for python.

Click 'Save' and then go to the left side main menu and click the **Tools** or the wrench icon.

![agent_builder3](../../images/agent_builder3.png)

### Create a Agent Tool

![agent_builder4](../../images/agent_builder4.png)

Once you are on Tools you will see that Google Cloud has one already ready for use which it the **'code-interpreter'** that allows users to perform various tasks, including: data analysis, data visualization, text processing, solving equations or optimization problems. We can guide our agent to use this tool any time the user asks to visualize a data set or ask how to write a python script. To learn more check Google Clouds documentation [here](https://cloud.google.com/vertex-ai/generative-ai/docs/extensions/code-interpreter).

For this tutorial we will be creating a tool that will connect to a data store. Our model will us this tool to answer scientific questions. This is essentially using Retrieval-Augmented Generation (RAG) a technique that allows the model to answer questions based on the data you provide it but by using agents we are allowing the model to work ....................

Click **'CREATE'** to create a new tool. Give your tool a name, then in the drop down menu under Type select **'Data store'**.

![agent_builder5](../../images/agent_builder5.png)

You will notice other fields have shown up, the options may differ if you already have a agent datastore set up.  

**Option A** If its your first time creating a data store tool you will probably see an option like the following. If you have this view, go ahead and click **'Create a data store'**.

![agent_builder6](../../images/agent_builder6.png)

**Option B** Otherwise you might see have this view if you already have a data store set up. If you have this view, click the drop down menu for 'Unstructured documents' and click the options that says **'Create a data store'**

![agent_builder7](../../images/agent_builder7.png)

Both options will open a new webpage and take you to the next step of this tutorial.

### Create a Data Store

**Note:** You may be prompted to configure your agent app if so enter the company name for your agent app. This can be anything you like. Set the location to **'US'**. Click **'Continue'**.

![agent_builder8](../../images/agent_builder8.png)

Next we'll create a data store that will allow us to connect a data source to our agent tool. Click **'CREATE DATA STORE'**.

![grounding4](../../images/grounding_4.png)

As you can see from the image below Google Cloud's Agent builder support the following data sources but for this tutorial we will use **'Cloud Storage'** as our data source.

![agent_builder9](../../images/agent_builder9.png)

Select **'Unstructured documents'** for the kind of data you are importing and then click **'BROWSE'**. Select the bucket that contains your files. Select **'CONTINUE'**.

![agent_builder10](../../images/agent_builder10.png)

Now you can give your datastore a name. Click **'DOCUMENT PROCESSING OPTIONS'** to enable chunking your data select **'Layout Parser'** as the default document parser and keep the chunk size limit to 500 tokens. Click **'CREATE'**. 

This will separate the text in your documents to a roughly 500 characters size chunks allowing only the relevant info to be presented to your model which will use less tokens and decrease the chance of your model hallucinating. 

![agent_builder11](../../images/agent_builder11.png)

You should see your data store listed as it being created. Select your data store and click **'CREATE'**.

![agent_builder12](../../images/agent_builder12.png)

### Add your data store to your tool

Give your data store some time to be created ~5 -10 mins. Now you can navigate back to the Agent Console (located in the previous web tab). Go back to Tools and Select **'Data store'** in the dropdown for Type.

Under Data stores click the drop down for **'Unstructured documents'** you should see your data store listed. Select your data store. 

**Note:** If you dont see your datastore listed click 'Refresh data store' or refresh your web page.

We can also add a **description** for this tool to explain the function for example: This tool will provide answers to scientific questions from the information in this data store.

![agent_builder13](../../images/agent_builder13.png)

The other fields shown are Grounding and summarization, they both can stay on their default setting.

- **Grounding:** You can set a score of how well you think the model will provide accurate information for the question you ask. If you score it as low then this will enable the model to rely on your data source for information rather than the data that is was pretrained on.

- **Summarization prompt:** You can add a prompt instructing the model how you would like to summarize the information it receives from the data store.

![agent_builder14](../../images/agent_builder14.png)

Click **'SAVE'**. 

### Adding our tools to the agent

Now we can navigate back to the Agents page on the left side menu. Once there we can add our tools and instructions on how and when to use our tools.in the example below we are referring to our tool using the `TOOL` function like so, `${Tool: NAME_OF_YOUR_TOOL}`. 

You can see from our example that we are listing instructions on what actions the model should take in a given situation:

1. Greet the user
2. Summarize the users request and confirm that the model understood what the user wanted
3. If the question is a scientific one it should obtain the related information from our data source tool
4. If the question is data related it should use the code interpreter tool
5. When the user is done thank them and say good bye

![agent_builder15](../../images/agent_builder15.png)

At the bottom of the form make sure your tools are checkmarked, then click **'SAVE'**.

![agent_builder16](../../images/agent_builder16.png)

### Testing out the agent

You have probably noticed that on the side of the screen there is a preview screen where you can test out your agent in the drop down menu for 'Agent' you can select the agent you would like to test the same can be said for the next drop down which is for the model you would like to test your agents on.

![agent_builder17](../../images/agent_builder17.png)

Now we can ask Gemini Pro, which is connected to our agent, some questions!

Try out some of these questions
- What is Nextflow
- Create a example Nextflow script
- What is long covid?
- What happens to people with diabetes and covid? 

You can see from the output below our agent was able to provide the correct information to Gemini Pro!


![agent_builder18](../../images/agent_builder18.png)
![agent_builder18](../../images/agent_builder19.png)

You can learn more on how you can implement agents to your use case by reading the documentation [here](https://cloud.google.com/dialogflow/cx/docs/basics).

## Conclusion

You have learned how to set up RAG with your own data and Google search on the console by creating a data store and a search app and enable grounding in the Vertex AI Studio playground.

## Clean up

Please remember to delete or stop your **Jupyter notebook** and delete your **data store** and **search app** on Agent Builder to prevent incurring charges. And if you have created any other services like buckets, please remember to delete them as well.